<a href='https://realpython.com/pytest-python-testing/#fixtures-managing-state-and-dependencies'>Real python</a>, nos aclara bien cual es el comportamiento de las fixtures:

*You can use the fixture by adding the function reference as an argument to your tests. Note that you don’t call the fixture function. pytest takes care of that. You’ll be able to use the return value of the fixture function as the name of the fixture function*

En esta parte voy a tratar de explicar el uso de fixtures en particular con este codigo:

In [ ]:
@pytest.fixture(scope="session")
def mkbox_params(): #aca vas a retornar la funcion maker que la podes modificar
    def _maker(
        *,
        seed=None,
        coordinates_scale=500,
        velocity_scale=220,
        mass_scale=12,
        size=1000,
    ):
        rng = np.random.default_rng(seed=seed)
        params = {
            "x": rng.uniform(0,coordinates_scale,size=size),
            "y": rng.uniform(0,coordinates_scale,size=size),
            "z": rng.uniform(0,coordinates_scale,size=size),
            "vx": rng.uniform(0,velocity_scale,size=size),
            "vy": rng.uniform(0,velocity_scale,size=size),
            "vz": rng.uniform(0,velocity_scale,size=size),
            "m": rng.uniform(0,mass_scale,size=size)
        }
        return params

    return _maker

@pytest.fixture(scope="session")
def mkbox(mkbox_params):
    def _maker(**kwargs):
        params = mkbox_params(**kwargs)
        return Box(**params)

    return _maker

El anterior es un codigo del voidfinder toolkit de la parte de testeo. Cuando nosotros ejecutamos pytest como en el ejemplo de abajo:

In [ ]:
def test_Box_initialization(mkbox):
    box = mkbox(seed=42, size=1000)# mkbox inmediatamente retorna _maker(**kwargs)
    #mkbox(seed=42, size=1000) --> _maker(**kwargs)
    assert box.x.unit == u.Mpc
    assert box.y.unit == u.Mpc
    assert box.z.unit == u.Mpc
    assert box.vx.unit == u.Mpc / u.h
    assert box.vy.unit == u.Mpc / u.h
    assert box.vz.unit == u.Mpc / u.h
    assert box.m.unit == u.M_sun
    assert len(box) == 1000
    assert repr(box) == "<Box size=1000>"
    assert isinstance(box.x, np.ndarray)
    assert isinstance(box.y, np.ndarray)
    assert isinstance(box.z, np.ndarray)
    assert isinstance(box.vx, np.ndarray)
    assert isinstance(box.vy, np.ndarray)
    assert isinstance(box.vz, np.ndarray)
    assert isinstance(box.m, np.ndarray)

Puede parecer extremadamente raro que esto ande, pero que sucede? En este contexto cuando vos llamas mkbox_params por ejemplo, este te retorna _maker y vos pensa una cosa, en el contexto de pytest cuando vos colocas mkbox_params vos estas llamando AL RESULTADO de esa funcion, no a la funcion.

Esto me produjo mucha confusion, pues yo antes habia armado:

In [ ]:
@pytest.fixture
def make_spherical_voids_params(**kwargs):

    params = {
        'n_voids':1000,
        'rad_scale': 15,
        'xyz_void_max_scale':500,
        'vel_xyz_void_max_scale':200,
        'min_delta':-0.95,
        'max_delta':-0.90,
        'min_poisson':-0.5,
        'max_poisson':0.5,
        'dtype': 0.5,
        'nran': 200,
        'seed':42
    }
    for key,value in kwargs.items():
        params[key] = value

    rng = np.random.default_rng(seed=params['seed'])
    void_params = {
        'rad' : rng.uniform(0,params['rad_scale'],params['n_voids']),
        'x_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
        'y_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
        'z_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
        'vel_x_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
        'vel_y_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
        'vel_z_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
        'delta' : rng.uniform(params['min_delta'],params['max_delta'],params['n_voids']),
        'poisson' : rng.uniform(params['min_poisson'], params['max_poisson'],params['n_voids']),
        'dtype' :  rng.uniform(0, params['dtype'],params['n_voids']),
        'nran' : rng.uniform(0, params['nran'],params['n_voids'])
    }
    return void_params
    


Entonces que sucedia? que yo queria hacer : spherical_voids_params = make_spherical_voids_params(n_voids=500) y lo que sucedia es que se me retornaba que 'dict object is not callable' Porque? Porque al llamar a esto en ralidad estaba retornando el diccionario void_params y no la funcion y a ese diccionario le estaba dando las variables: void_params(n_voids=500) dandome el error. Cual es la solucion?

In [ ]:
@pytest.fixture
def make_spherical_voids_params():
    #aca vas a retornar un valor, no una funcion
    def _maker(**kwargs):
        params = {
            'n_voids':1000,
            'rad_scale': 15,
            'xyz_void_max_scale':500,
            'vel_xyz_void_max_scale':200,
            'min_delta':-0.95,
            'max_delta':-0.90,
            'min_poisson':-0.5,
            'max_poisson':0.5,
            'dtype': 0.5,
            'nran': 200,
            'seed':42
        }
        for key,value in kwargs.items():
            params[key] = value

        rng = np.random.default_rng(seed=params['seed'])
        void_params = {
            'rad' : rng.uniform(0,params['rad_scale'],params['n_voids']),
            'x_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
            'y_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
            'z_void' : rng.uniform(0,params['xyz_void_max_scale'],params['n_voids']),
            'vel_x_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
            'vel_y_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
            'vel_z_void' : rng.uniform(0,params['vel_xyz_void_max_scale'],params['n_voids']),
            'delta' : rng.uniform(params['min_delta'],params['max_delta'],params['n_voids']),
            'poisson' : rng.uniform(params['min_poisson'], params['max_poisson'],params['n_voids']),
            'dtype' :  rng.uniform(0, params['dtype'],params['n_voids']),
            'nran' : rng.uniform(0, params['nran'],params['n_voids'])
        }
        return void_params
    
    return _maker

La solucion es hacer que make_spherical_voids_params retorne una funcion, entonces cuando vos haces make_spherical_voids_params(n_voids=500) en realidad estas llamando a _maker(n_voids=500)